# First version. Net.

> First version of constructor.

In [1]:
#hide
import torch
import torch.nn as nn

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [2]:
# hide
from collections import OrderedDict
from functools import partial


from model_constructor.layers import ConvLayer, Flatten, SEBlock, SimpleSelfAttention, noop

In [3]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

# ResBlock

In [4]:
from model_constructor.net import ResBlock

In [5]:
#hide_input
print_doc(ResBlock)

<h4 id="ResBlock" class="doc_header"><code>class</code> <code>ResBlock</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L25" class="source_link" style="float:right">[source]</a></h4>

> <code>ResBlock</code>(**`expansion`**, **`ni`**, **`nh`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **`se`**=*`False`*, **`se_reduction`**=*`16`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*) :: `Module`

Resnet block

In [6]:
ResBlock(1,64,64,sa=True)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [7]:
ResBlock(4,64,64,sa=True, dw=True)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [8]:
ResBlock(4,64,64,sa=True, groups=4)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [9]:
ResBlock(2,64,64,act_fn=nn.LeakyReLU(), bn_1st=False)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [10]:
ResBlock(2, 64, 64, sa=True, se=True)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEBlock(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=8, out_features=128, bias=True)

# NewResBlock

NewResBlock now is YaResBlock - Yet Another ResNet Block! It is now  at model_constructor.yaresnet.
Here i left old name for compatibility with existing Notebooks.

In [11]:
from model_constructor.net import NewResBlock

In [12]:
#hide_input
print_doc(NewResBlock)

<h4 id="NewResBlock" class="doc_header"><code>class</code> <code>NewResBlock</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L64" class="source_link" style="float:right">[source]</a></h4>

> <code>NewResBlock</code>(**`expansion`**, **`ni`**, **`nh`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **`se`**=*`False`*, **`se_reduction`**=*`16`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*) :: `Module`

YaResnet block.
This is first impl, deprecated, use yaresnet module.

In [13]:
NewResBlock(4, 64, 128, dw=1)

NewResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): ReLU(inplace=True)
)

In [14]:
#hide
bl = NewResBlock(1,64,64,sa=True)
bl

NewResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [15]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [16]:
#hide
bl = NewResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False)
bl

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [17]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [18]:
#hide
bl = NewResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False, groups=4)
bl

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [19]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


# Stem, Body, Head

In [20]:
from model_constructor.net import _make_stem, _make_layer, _make_body, _make_head

In [22]:
#hide_input
print_doc(_make_stem)
print_doc(_make_layer)
print_doc(_make_body)
print_doc(_make_head)


<h4 id="_make_stem" class="doc_header"><code>_make_stem</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L103" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_stem</code>()



<h4 id="_make_layer" class="doc_header"><code>_make_layer</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L115" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_layer</code>(**`expansion`**, **`ni`**, **`nf`**, **`blocks`**, **`stride`**, **`sa`**)



<h4 id="_make_body" class="doc_header"><code>_make_body</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L126" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_body</code>()



<h4 id="_make_head" class="doc_header"><code>_make_head</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L135" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_head</code>()



# Net - Model Constructor.

In [23]:
from model_constructor.net import Net

In [24]:
#hide_input
print_doc(Net)

<h4 id="Net" class="doc_header"><code>class</code> <code>Net</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/net.py#L142" class="source_link" style="float:right">[source]</a></h4>

> <code>Net</code>(**`name`**=*`'Net'`*, **`c_in`**=*`3`*, **`c_out`**=*`1000`*, **`block`**=*`ResBlock`*, **`conv_layer`**=*`ConvLayer`*, **`block_sizes`**=*`[64, 128, 256, 512]`*, **`layers`**=*`[2, 2, 2, 2]`*, **`norm`**=*`BatchNorm2d`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`expansion`**=*`1`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`sa`**=*`False`*, **`se`**=*`False`*, **`se_reduction`**=*`16`*, **`bn_1st`**=*`True`*, **`zero_bn`**=*`True`*, **`stem_stride_on`**=*`0`*, **`stem_sizes`**=*`[32, 32, 64]`*, **`stem_pool`**=*`MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)`*, **`stem_bn_end`**=*`False`*, **`_init_cnn`**=*`init_cnn`*, **`_make_stem`**=*`_make_stem`*, **`_make_layer`**=*`_make_layer`*, **`_make_body`**=*`_make_body`*, **`_make_head`**=*`_make_head`*)

Model constructor. As default - xresnet18.
First version, still here for compatibility. Use ModelConstructor instead.

In [25]:
model  = Net()
model

Net constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [26]:
model._block_sizes

[64, 128, 256, 512]

In [27]:
model.block_sizes

[64, 64, 128, 256, 512]

In [28]:
model._block_sizes = [128, 256, 512, 1024]
model

Net constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [128, 256, 512, 1024]
  layers: [2, 2, 2, 2]

In [29]:
model.block_sizes

[64, 128, 256, 512, 1024]

In [30]:
#hide
model  = Net(stem_sizes=[3,32,32,64])
model

Net constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [31]:
model.block_sizes

[64, 64, 128, 256, 512]

In [32]:
model = Net()

In [33]:
#hide
# model.block_sizes = [64, 128, 256, 512] # wrong way --> use _block_sizes
# model

In [34]:
#collapse_output
model.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [35]:
#collapse_output
model.stem_stride_on = 1
model.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [36]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


/home/jzz/anaconda3/envs/mc_dev/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [37]:
model.bn_1st = False

In [38]:
model.act_fn =nn.LeakyReLU(inplace=True)

In [39]:
model.sa = True
model.se = True

In [40]:
#collapse_output
model.body.l_1

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEBlock(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
          (se_act): ReLU(inplace=True)
          (fc_expand): Linear(in_features=8, out_features=128, bias=True)
          (sigmoid): Sigmoid()
        )
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLaye

In [41]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [42]:
#hide
model.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEBlock(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=64, out_features=4, bias=True)
          (se_act): ReLU(inplace=True)
          (fc_expand): Linear(in_features=4, out_features=64, bias=True)
          (sigmoid): Sigmoid()
        )
      )
    )
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): ResBlock(
    

In [43]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [44]:
#hide
model.groups = 4
model.expansion = 4
model.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEBlock(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=256, out_features=1

In [45]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [46]:
#hide
model.groups = 1
model.dw = True
model.expansion = 4
model.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEBlock(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=256, out_features=

In [47]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [48]:
#hide
model.groups = 1
model.dw = 0
model.expansion = 1

In [49]:
model.block = NewResBlock

In [50]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = model.body.l_1.bl_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 16, 16]), f"size"

torch.Size([16, 128, 16, 16])


In [51]:
#hide
model.body.l_1.bl_0

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEBlock(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=8, out_features=128, bias=True)
        (sigmoid): Sigmoid()
      )
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1,

In [52]:
# model = Net(expansion=4)
model.expansion = 4

In [53]:
#hide
model.stem_bn_end = True

In [54]:
#hide
model.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [55]:
#hide
model.body.l_1.bl_0

NewResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEBlock(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=512, out_features=32,

In [56]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = model.body.l_1.bl_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [57]:
#hide
model.stem_bn_end = False

In [58]:
#collapse_input
m = model()

In [59]:
#hide
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0)

In [60]:
#collapse_output
m.body

Sequential(
  (l_0): Sequential(
    (bl_0): NewResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (se): SEBlock(
          (squeeze): AdaptiveAvgPool2d(output_size=1)
          (excitation): Sequential(
         

In [61]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = m(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


# end
model_constructor
by ayasyrev